# Breaching privacy

This notebook does the same job as the cmd-line tool `simulate_breach.py`, but also directly visualizes the user data and reconstruction

In [1]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()
breaching.utils.huggingface_offline_mode(True)

### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

Choose `case/data=` `shakespeare`, `wikitext`over `stackoverflow` here:

In [2]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=["case/data=wikitext", "case/server=malicious-transformer",
                                                      "case.model=gpt2",
                                                      "attack=decepticon"])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
          
# device = torch.device(f'cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

Investigating use case single_imagenet with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [3]:
cfg.case.user.num_data_points = 8 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [512] # This is the sequence length

cfg.case.model = "gpt2S" #+ "gpt2" #"transformer3"
cfg.case.server.provide_public_buffers = True

cfg.case.server.has_external_data = True
cfg.case.data.tokenizer = "gpt2"

cfg.attack.token_strategy="embedding-norm"
cfg.case.server.param_modification.v_length = 32

cfg.case.server.param_modification.eps = 1e-8
cfg.case.server.param_modification.imprint_sentence_position = 0
cfg.case.server.param_modification.softmax_skew = 100000000
cfg.case.server.param_modification.sequence_token_weight = 1

cfg.case.server.param_modification.measurement_scale = 1

cfg.case.server.pretrained = False

### Instantiate all parties

In [4]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Model architecture gpt2S loaded with 124,439,808 parameters and 12,582,924 buffers.
Overall this is a data ratio of   30381:1 for target shape [8, 512] give

### Simulate an attacked FL protocol

True user data is returned only for analysis

In [5]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is -0.06816399842500687, feature std is 0.707169234752655.
Computing user update in model mode: eval.


In [6]:
# user.print(true_user_data)

# Reconstruct user data

In [7]:
#attacker.cfg.sentence_algorithm="k-means"
# attacker.embedding_token_weight = 0.25

In [8]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

# user.print(reconstructed_user_data)

Recovered tokens tensor([   11,    11,    11,  ..., 50203, 50210, 50210]) through strategy embedding-norm.
Recovered 3868 embeddings with positional data from imprinted layer.
Assigned [479, 489, 484, 423, 512, 487, 482, 512] breached embeddings to each sentence.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/accuracy/3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Thu Jan  6 18:45:02 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/bleu/8bc44c0b5bba429a0a448e9968a8961bf9479d6ee30e00e93fed2c693040265a (last modified on Thu Jan  6 18:45:03 2022) since it couldn't be found locally at bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/m

In [9]:
results = dict()
for sentence_alg in ["dynamic-threshold"]:
    for embedding_weight in [0, 1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.5, 0.75, 0.95, 1.0, 10]:
        attacker.cfg.sentence_algorithm=sentence_alg
        attacker.cfg.embedding_token_weight = embedding_weight
        print(f"Evaluating {sentence_alg} with {embedding_weight}")
        try:
            reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                          server.secrets, dryrun=cfg.dryrun)

            metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                                server.model, cfg_case=cfg.case, setup=setup)
            results[f"{sentence_alg}-{embedding_weight}"] = metrics["accuracy"]
        except:
            results[f"{sentence_alg}-{embedding_weight}"] = 0.0

Evaluating dynamic-threshold with 0
Recovered tokens tensor([   11,    11,    11,  ..., 50203, 50210, 50210]) through strategy embedding-norm.
Recovered 3858 embeddings with positional data from imprinted layer.
Assigned [476, 486, 438, 492, 512, 483, 491, 480] breached embeddings to each sentence.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/accuracy/3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Thu Jan  6 18:45:02 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/bleu/8bc44c0b5bba429a0a448e9968a8961bf9479d6ee30e00e93fed2c693040265a (last modified on Thu Jan  6 18:45:03 2022) since it couldn't be found locally at bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/hu

Evaluating dynamic-threshold with 0.1
Recovered tokens tensor([   11,    11,    11,  ..., 50203, 50210, 50210]) through strategy embedding-norm.
Recovered 3858 embeddings with positional data from imprinted layer.
Assigned [476, 486, 438, 492, 512, 483, 491, 480] breached embeddings to each sentence.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/accuracy/3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Thu Jan  6 18:45:02 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/bleu/8bc44c0b5bba429a0a448e9968a8961bf9479d6ee30e00e93fed2c693040265a (last modified on Thu Jan  6 18:45:03 2022) since it couldn't be found locally at bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/

Replaced token 1057 with corr 0.08473658561706543 with new token with corr 0.22825545072555542
Replaced token 1074 with corr 0.08387237042188644 with new token with corr 0.1788710653781891
Replaced token 1089 with corr 0.09365814179182053 with new token with corr 0.23969314992427826
Replaced token 1093 with corr 0.0979638621211052 with new token with corr 0.25052905082702637
Replaced token 1098 with corr 0.07346440106630325 with new token with corr 0.14864100515842438
Replaced token 1153 with corr 0.10632549226284027 with new token with corr 0.2262493073940277
Replaced token 1171 with corr 0.09213504195213318 with new token with corr 0.23409555852413177
Replaced token 1240 with corr 0.1078638881444931 with new token with corr 0.255260169506073
Replaced token 1261 with corr 0.1271013468503952 with new token with corr 0.2786852717399597
Replaced token 1263 with corr 0.11042303591966629 with new token with corr 0.23266048729419708
Replaced token 1264 with corr 0.12782299518585205 with new

Replaced token 3480 with corr 0.11261890828609467 with new token with corr 0.23385603725910187
Replaced token 3486 with corr 0.094296395778656 with new token with corr 0.26696982979774475
Replaced token 3504 with corr 0.10243967920541763 with new token with corr 0.21259687840938568
Replaced token 3508 with corr 0.11241219192743301 with new token with corr 0.3117140233516693
Replaced token 3522 with corr 0.11608175188302994 with new token with corr 0.2361796647310257
Replaced token 3540 with corr 0.0735185369849205 with new token with corr 0.18384917080402374
Replaced token 3547 with corr 0.12320207059383392 with new token with corr 0.2969287633895874
Replaced token 3565 with corr 0.07344812899827957 with new token with corr 0.14897918701171875
Replaced token 3573 with corr 0.0852590948343277 with new token with corr 0.22231847047805786
Replaced token 3583 with corr 0.07468339055776596 with new token with corr 0.15685757994651794
Replaced token 3611 with corr 0.1045120358467102 with new

Replaced token 490 with corr 0.09709368646144867 with new token with corr 0.31151992082595825
Replaced token 511 with corr 0.07224166393280029 with new token with corr 0.14823205769062042
Replaced token 525 with corr 0.11561029404401779 with new token with corr 0.25178301334381104
Replaced token 538 with corr 0.09454026818275452 with new token with corr 0.15348148345947266
Replaced token 566 with corr 0.11152159422636032 with new token with corr 0.22072097659111023
Replaced token 572 with corr 0.10247683525085449 with new token with corr 0.1441391110420227
Replaced token 592 with corr 0.09372688829898834 with new token with corr 0.31124797463417053
Replaced token 599 with corr 0.09186166524887085 with new token with corr 0.20139718055725098
Replaced token 615 with corr 0.14499571919441223 with new token with corr 0.2901389002799988
Replaced token 618 with corr 0.11568886786699295 with new token with corr 0.23867349326610565
Replaced token 640 with corr 0.12705056369304657 with new toke

...

Replaced token 3728 with corr 0.34511834383010864 with new token with corr 0.34511837363243103
Replaced token 3729 with corr 0.2955830991268158 with new token with corr 0.2955830991268158
Replaced token 3730 with corr 0.2819732427597046 with new token with corr 0.2819732129573822
Replaced token 3731 with corr 0.22080348432064056 with new token with corr 0.22080348432064056
Replaced token 3732 with corr 0.28329312801361084 with new token with corr 0.28329309821128845
Replaced token 3733 with corr 0.29225462675094604 with new token with corr 0.29225459694862366
Replaced token 3734 with corr 0.2669542133808136 with new token with corr 0.2669542133808136
Replaced token 3735 with corr 0.11797637492418289 with new token with corr 0.2217080146074295
Replaced token 3736 with corr 0.09470909088850021 with new token with corr 0.21329985558986664
Replaced token 3737 with corr 0.1390988528728485 with new token with corr 0.2615695893764496
Replaced token 3738 with corr 0.368228554725647 with new to

Replaced token 3816 with corr 0.2988011837005615 with new token with corr 0.2988011837005615
Replaced token 3817 with corr 0.19687430560588837 with new token with corr 0.19687432050704956
Replaced token 3818 with corr 0.1288331151008606 with new token with corr 0.23295429348945618
Replaced token 3819 with corr 0.2907969057559967 with new token with corr 0.2907968759536743
Replaced token 3820 with corr 0.27051809430122375 with new token with corr 0.27051809430122375
Replaced token 3821 with corr 0.28557437658309937 with new token with corr 0.28557437658309937
Replaced token 3822 with corr 0.16054946184158325 with new token with corr 0.16640932857990265
Replaced token 3823 with corr 0.22404614090919495 with new token with corr 0.22404614090919495
Replaced token 3824 with corr 0.2825174331665039 with new token with corr 0.2825174033641815
Replaced token 3825 with corr 0.246450275182724 with new token with corr 0.2464502453804016
Replaced token 3826 with corr 0.2959827780723572 with new to

Replaced token 3904 with corr 0.25841519236564636 with new token with corr 0.25841519236564636
Replaced token 3905 with corr 0.32950228452682495 with new token with corr 0.32950225472450256
Replaced token 3906 with corr 0.2750011086463928 with new token with corr 0.2750011086463928
Replaced token 3907 with corr 0.2930745780467987 with new token with corr 0.2930745780467987
Replaced token 3908 with corr 0.33435431122779846 with new token with corr 0.33435431122779846
Replaced token 3909 with corr 0.3485250174999237 with new token with corr 0.3485250174999237
Replaced token 3910 with corr 0.2652069330215454 with new token with corr 0.2652069330215454
Replaced token 3911 with corr 0.2971383035182953 with new token with corr 0.2971383333206177
Replaced token 3912 with corr 0.28748977184295654 with new token with corr 0.28748977184295654
Replaced token 3913 with corr 0.3032384514808655 with new token with corr 0.3032384514808655
Replaced token 3914 with corr 0.08463185280561447 with new tok

Replaced token 3992 with corr 0.20836299657821655 with new token with corr 0.20836301147937775
Replaced token 3993 with corr 0.2797645926475525 with new token with corr 0.2797646224498749
Replaced token 3994 with corr 0.24389046430587769 with new token with corr 0.24389047920703888
Replaced token 3995 with corr 0.2726772427558899 with new token with corr 0.2726772427558899
Replaced token 3996 with corr 0.2776162028312683 with new token with corr 0.2776161730289459
Replaced token 3997 with corr 0.28997802734375 with new token with corr 0.2899779975414276
Replaced token 3998 with corr 0.24073143303394318 with new token with corr 0.24073141813278198
Replaced token 3999 with corr 0.25683411955833435 with new token with corr 0.25683411955833435
Replaced token 4000 with corr 0.33340635895729065 with new token with corr 0.33340638875961304
Replaced token 4001 with corr 0.3169530928134918 with new token with corr 0.3169530928134918
Replaced token 4002 with corr 0.3041837811470032 with new toke

Replaced token 4080 with corr 0.27061501145362854 with new token with corr 0.27061501145362854
Replaced token 4081 with corr 0.09662853181362152 with new token with corr 0.19235314428806305
Replaced token 4082 with corr 0.358834832906723 with new token with corr 0.3588348627090454
Replaced token 4083 with corr 0.3017421364784241 with new token with corr 0.3017421066761017
Replaced token 4084 with corr 0.2358466535806656 with new token with corr 0.2358466237783432
Replaced token 4085 with corr 0.28542622923851013 with new token with corr 0.28542619943618774
Replaced token 4086 with corr 0.32203593850135803 with new token with corr 0.32203593850135803
Replaced token 4087 with corr 0.2609933912754059 with new token with corr 0.26099342107772827
Replaced token 4088 with corr 0.3617456257343292 with new token with corr 0.3617456257343292
Replaced token 4089 with corr 0.29691386222839355 with new token with corr 0.29691386222839355
Replaced token 4090 with corr 0.298216313123703 with new tok

In [10]:
results

{'dynamic-threshold-0': 0.865234375,
 'dynamic-threshold-0.0001': 0.865234375,
 'dynamic-threshold-0.001': 0.865234375,
 'dynamic-threshold-0.01': 0.865234375,
 'dynamic-threshold-0.1': 0.865234375,
 'dynamic-threshold-0.2': 0.865234375,
 'dynamic-threshold-0.5': 0.891845703125,
 'dynamic-threshold-0.75': 0.904541015625,
 'dynamic-threshold-0.95': 0.90234375,
 'dynamic-threshold-1.0': 0.90087890625,
 'dynamic-threshold-10': 0.90087890625}

In [13]:
attacker.cfg.sentence_algorithm="dynamic-threshold"
attacker.cfg.embedding_token_weight = 0

In [14]:
results_server = dict()
for v_length in [8, 16, 32, 48, 64]:
    for eps in [10, 1, 0.1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]:
        for softmax_skew in [1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9, 1e10, 1e11, 1e12]:
            for sequence_token_weight in [10, 1, 0.1, 0.01, 1e-3]:
                for measurement_scale in [1e3, 1e2, 1, 0.1, 1e-2, 1e-3]:
                    key = f"{v_length}-{eps}-{softmax_skew}-{sequence_token_weight}-{measurement_scale}"
                    print(f"Checking {key}")
                    cfg.attack.token_strategy="embedding-norm"
                    cfg.case.server.param_modification.v_length = v_length

                    cfg.case.server.param_modification.eps = eps
                    cfg.case.server.param_modification.imprint_sentence_position = 0
                    cfg.case.server.param_modification.softmax_skew = softmax_skew
                    cfg.case.server.param_modification.sequence_token_weight = sequence_token_weight

                    cfg.case.server.param_modification.measurement_scale = measurement_scale

                    cfg.case.server.pretrained = False

                    user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
                    attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)

                    server_payload = server.distribute_payload()
                    shared_data, true_user_data = user.compute_local_updates(server_payload)
                    try:
                        reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                                      server.secrets, dryrun=cfg.dryrun)

                        metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                                            server.model, cfg_case=cfg.case, setup=setup)

                        results_server[key] = metrics["accuracy"]
                    except:
                        results_server[key] = 0.0

Checking 8-10-10.0-10-1000.0
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external 

Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/google_bleu/bd7e6c5887f75974a6272946388ed86b5be85e95466b9f71edd4bd0ff5041640 (last modified on Thu Jan  6 18:45:22 2022) since it couldn't be found locally at google_bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/sacrebleu/015a527b2e180e7da27bda77c3fb309ebab15dba5d484fa6f3fec5e9f762c849 (last modified on Thu Jan  6 18:45:40 2022) since it couldn't be found locally at sacrebleu, or remotely on the Hugging Face Hub.
METRICS: | Accuracy: 0.0681 | S-BLEU: 0.09 | FMSE: 8.6089e-03 | 
 G-BLEU: 0.12 | ROUGE1: 0.41| ROUGE2: 0.05 | ROUGE-L: 0.17| Token Acc: 92.11% | Label Acc: 92.11%
Checking 8-10-10.0-10-0.01
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a49

Recovered tokens tensor([   11,    11,    11,  ..., 49658, 50210, 50210]) through strategy embedding-norm.
Recovered 340 embeddings with positional data from imprinted layer.
Cannot separate 8 seeds by thresholding!
Could assemble only 2 seeds searching on threshold 0.0009999999999998899.
Filling with 6 random seeds...These sentences will be scrambled.
Assigned [26, 309, 1, 1, 1, 2] breached embeddings to each sentence.
Checking 8-10-10.0-1-100.0
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a2

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is 0.0004966752021573484, feature std is 0.009933419525623322.
Computing user update in model mode: eval.
Recovered tokens tensor([   11,    11,    11,  ..., 50210, 50210, 50210]) through strategy embedding-norm.
Recovered 3837 embeddings with positional data from imprinted layer.
Assigned [

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is 1.8897814750671387, feature std is 104.6097183227539.
Computing user update in model mode: eval.
Recovered tokens tensor([   11,    11,    11,  ..., 50210, 50210, 50210]) through strategy embedding-norm.
Recovered 587 embeddings with positional data from imprinted layer.
Assigned [60, 62, 6, 119, 62, 13, 29, 236] breached embeddings to each sentence.
Checking 8-10-10.0-0.1-1
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext

...

Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is 5.518501281738281, feature std is 98.56505584716797.
Computing user update in model mode: eval.
Recovered tokens tensor([   11,    11,    11,  ..., 50203, 50210, 50210]) through strategy embedding-norm.
Recovered 3463 embeddings with positional data from imprinted layer.
Assigned [512, 335, 512, 512, 253, 512, 315, 512] breached embeddings to each sentence.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/accuracy/3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Thu Jan  6 18:45:02 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/bleu/8bc44c0b5bba429a0a448e9968a8961bf9479d6ee30e00e93fed2

METRICS: | Accuracy: 0.5664 | S-BLEU: 0.22 | FMSE: 7.3132e-03 | 
 G-BLEU: 0.24 | ROUGE1: 0.57| ROUGE2: 0.26 | ROUGE-L: 0.53| Token Acc: 68.21% | Label Acc: 68.21%
Checking 8-0.1-10000.0-0.01-0.01
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210b

Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is 2.7736408710479736, feature std is 101.084716796875.
Computing user update in model mode: eval.
Recovered tokens tensor([   11,    11,    11,  ..., 50203, 50210, 50210]) through strategy embedding-norm.
Recovered 3415 embeddings with positional data from imprinted layer.
Assigned [392, 400, 428, 361, 457, 424, 441, 512] breached embeddings to each sentence.
Using the latest cached version of the module from /home/jonas/.cache

Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/sacrebleu/015a527b2e180e7da27bda77c3fb309ebab15dba5d484fa6f3fec5e9f762c849 (last modified on Thu Jan  6 18:45:40 2022) since it couldn't be found locally at sacrebleu, or remotely on the Hugging Face Hub.
METRICS: | Accuracy: 0.6077 | S-BLEU: 0.27 | FMSE: 7.6017e-03 | 
 G-BLEU: 0.28 | ROUGE1: 0.61| ROUGE2: 0.31 | ROUGE-L: 0.58| Token Acc: 68.24% | Label Acc: 68.24%
Checking 8-0.1-10000.0-0.001-0.01
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Using the latest cached version of the module from /home/jona

KeyboardInterrupt: 

In [17]:
max(results_server.values())

0.735595703125

In [18]:
results_server

{'8-10-10.0-10-1000.0': 0.0,
 '8-10-10.0-10-100.0': 0.0,
 '8-10-10.0-10-1': 0.0224609375,
 '8-10-10.0-10-0.1': 0.068115234375,
 '8-10-10.0-10-0.01': 0.06396484375,
 '8-10-10.0-10-0.001': 0.052978515625,
 '8-10-10.0-1-1000.0': 0.0,
 '8-10-10.0-1-100.0': 0.0,
 '8-10-10.0-1-1': 0.022216796875,
 '8-10-10.0-1-0.1': 0.058837890625,
 '8-10-10.0-1-0.01': 0.077880859375,
 '8-10-10.0-1-0.001': 0.05419921875,
 '8-10-10.0-0.1-1000.0': 0.0,
 '8-10-10.0-0.1-100.0': 0.0,
 '8-10-10.0-0.1-1': 0.0205078125,
 '8-10-10.0-0.1-0.1': 0.1025390625,
 '8-10-10.0-0.1-0.01': 0.083251953125,
 '8-10-10.0-0.1-0.001': 0.052490234375,
 '8-10-10.0-0.01-1000.0': 0.0,
 '8-10-10.0-0.01-100.0': 0.0,
 '8-10-10.0-0.01-1': 0.020751953125,
 '8-10-10.0-0.01-0.1': 0.055419921875,
 '8-10-10.0-0.01-0.01': 0.08251953125,
 '8-10-10.0-0.01-0.001': 0.07763671875,
 '8-10-10.0-0.001-1000.0': 0.0,
 '8-10-10.0-0.001-100.0': 0.0,
 '8-10-10.0-0.001-1': 0.02197265625,
 '8-10-10.0-0.001-0.1': 0.09228515625,
 '8-10-10.0-0.001-0.01': 0.05151367

In [9]:
text = """
Decepticon
From Wikipedia, the free encyclopedia
Jump to navigationJump to search
For other uses, see Decepticon (disambiguation).
"Deceptacon" redirects here. For other uses, see Deceptacon (disambiguation).
Not to be confused with Deception (disambiguation).

This article has multiple issues. Please help improve it or discuss these issues on the talk page. (Learn how and when to remove these template messages)
This article needs additional citations for verification. (March 2015)
This article may contain an excessive amount of intricate detail that may interest only a particular audience. (March 2018)
Decepticon
Decepticon small.jpg
Decepticon Insignia
Publication information
Publisher	Marvel Comics, IDW Publishing, Dreamwave Productions, Devil's Due Publishing, Fun Publications
First appearance	The Transformers #1
(September 1984)
In-story information
Base(s)	Homeworld: Kaon, Cybertron
Other: Chaar, the Victory (G1), the Megastar (RID), Decepticon City (G1), the Nemesis (G1, ROTF, Animated, SG, Prime).
Leader(s)	Megatron/Galvatron (stated in exodus and the cartoon), Starscream, Steeljaw, Liege Maximo, Unicron, The Fallen/Megatronus (in the movie series)
The Decepticons are the main antagonists in the fictional universes of the Transformers multimedia franchise.[1] They are depicted as a faction of sentient robotic lifeforms led by Megatron, identified by a purple face-like insignia. Capable of transforming into alternate forms, these are often high-tech or powerful vehicles; including aircraft, military vehicles, construction vehicles, expensive luxury or sports cars and even smaller-than-human-sized objects.

Decepticons are Cybertronian descendants of Terrakors, who are a fictional species of sentient self-configuring modular robotic lifeforms from Skalorr. They are a synergistic blend of biological evolution and technological engineering. The exotic materials that make up their composition is a living metal with a self-replicating cellular structure and genetic code.

In the Japanese version of the franchise, the Decepticons are called Destron[2] or Deathtron (Japanese: デストロン Desutoron). The only exception to this naming convention is Car Robots, where the sub-group referred to as "Decepticons" in the Robots in Disguise adaptation, is known in Japan as the Combatrons (the Japanese name of the G1 subgroup known as the Combaticons). The overall name of the Robots in Disguise villain faction is Destronger.[citation needed][clarification needed] However, the Japanese Version of Transformers Animated uses Decepticon (ディセプティコン).

As opposed to the Autobots' Supreme Commander, the Primes, the Decepticons' highest ranking leader is often given the title Emperor of Destruction. Beginning with the original Generation 1 cartoon, the Decepticon rallying cry has been "Decepticons attack!", as well as "Transform and rise up!" in Transformers: Animated as a play on the Autobots' "Transform and roll out!" rallying cry.


Contents
1	Transformers: Generation 1
1.1	Members
1.2	Hasbro toyline
1.3	Marvel Comics
1.4	Animated series
1.5	Dreamwave Productions
1.6	IDW Publishing
1.7	Beast Era
2	Transformers: Robots in Disguise
3	Unicron Trilogy
4	Live-action films
4.1	Transformers (2007 film)
4.2	Transformers: Revenge of the Fallen (2009 film)
4.3	Transformers: Dark of the Moon (2011 film)
4.4	Transformers: Age of Extinction (2014 film)
4.5	Transformers: The Last Knight (2017 film)
4.6	Bumblebee (2018 film)
5	Transformers Animated
6	Transformers: Timelines
7	Aligned Continuity
7.1	Transformers: War for Cybertron
7.2	Transformers: Prime
7.3	Transformers: Robots in Disguise
8	See also
9	References
Transformers: Generation 1
The Decepticons are from a race called Transformers - robots that can change into vehicles, mechanical devices and even animal forms. They waged war for eons on their home planet of Cybertron against the heroic Autobots. Both factions needed supplies, and select numbers from each side intended to get them. The Autobots did not wish to fight the Decepticons, but they fought in space anyway. When the Decepticons board the Autobots' ship it takes a beating and crashes onto Primitive Earth and knocks everyone unconscious. Millions of years later, in 1984 a volcano eruption causes a probe to give them all new forms which come from vehicles in the surrounding area. After the featured film The Transformers: The Movie, Megatron was reformed as Galvatron, and Optimus Prime was replaced for a time by Rodimus Prime, only to return later on.

Members
Unlike the Autobots, whose leader is a Prime bearing a Matrix, the Decepticons are led by the most powerful of their ranks (usually Megatron). This tends to cause some conflict, given how generally every Decepticon thinks that they're the most powerful. Also, the Decepticons are not exactly the most compassionate beings in the universe, but not all fight for greed. More than a few have a sense of honor, while others believe that Cybertron would be better protected by aggressive expansion. Ultimately, the Decepticons desire to protect their homeworld, even if it is at the expense of others. Other leaders include Galvatron, Shockwave, Soundwave, Scorponok, Devil Z, Deathsaurous, Starscream, Thunderwing, Jiaxus, Ratbat, and Bludgeaon. Some continuities also has Liege Maximo and The Fallen as the ancestor of the Decepticon faction.

Hasbro toyline
Hasbro launched the Transformers toy line with eighteen different Autobot characters, all of whom transformed into automobiles, while the ten distinct Decepticons, (seven packages as three came two to a box/pack) were weapons, aircraft and communications equipment. Many of the Transformers were based on Takara designs. Optimus Prime was among the first Transformers released from Hasbro in 1984. The character listing/mini-poster that came inside Transformer packaging identified him as "Autobot Commander", as contrasted with Megatron's title of "Decepticon Leader".

Marvel Comics
Main article: The Transformers (Marvel Comics)
In the Marvel comic continuity, it was established that the Decepticons came in and out of the gladiator pits on Cybertron. Early on, the leadership of the Decepticons constantly changed due to their tendency to kill and replace their leaders. It would not be until a lowly Decepticon affiliated gladiator named Megatron assumed the throne that the Decepticons became a true threat to the rest of the population of Cybertron.

While Megatron was the primary leader, other Decepticons had prominent roles as leaders of the group. Shockwave, Scorponok, Bludgeon, and Ratbat all led the Decepticons, often overthrowing Megatron in the process or taking over the group during the period when Megatron was believed dead.

In the G2 Marvel series (which continues from the G1 Marvel series), it is stated that the Decepticon forces on Cybertron underwent a massive split in the wake of Megatron's disappearance when Megatron, Optimus Prime, and their respective troops crashed on Earth. The smartest, strongest, and most intelligent Decepticons left Cybertron to colonize nearby galaxies, leaving a token leadership of overly cruel and duplicitous Decepticons to control Cybertron. These Decepticons who left their homeworld became "The Second Generation" Decepticons, who were led by the villainous Liege Maximo. On the final page of the last issue, the Liege Maximo indicates that the Decepticons and all their descendants came from him, which indicates that he is the original founder of the Decepticons.

Animated series
The Decepticons are a malevolent race of robot warriors, brutal and merciless. The Decepticons are driven by a single undeviating goal: total domination of the universe. In the war that raged between the Decepticons and Autobots for millions of years, their home planet of Cybertron was drained of its once rich sources of energy. In hopes of finding new reserves for making Energon, their basic fuel, the Decepticons followed the Autobots to Earth. Under the leadership of the ruthless Megatron, they continued their efforts to destroy the remaining Autobot forces. They attacked throughout the Earth and in space, using their underwater Decepticon headquarters as their staging base and bunker. But, by the year 2000, the Decepticons had proven to be the losers of the last Great War. In an attempt to bolster the Decepticons' strength, the evil planet gobbling world, Unicron, reformed Megatron into a new robot named Galvatron to lead the Decepticons in a new era. Now, in 2006, the Decepticons have retreated to a burned out hulk of a planet called Chaar, a world of ash and ruin. Galvatron and his Decepticon lieutenants, the sleek and awesome Cyclonus, and the mighty Scourge, the master of the dreaded Sweeps, strike fear throughout the universe. Their ultimate goal: to reconquer Cybertron and destroy the Autobots.

In the original animated series continuity, the Decepticons owe their warlike ways to their faction's origin as military hardware robots, created by the five-faced aliens, the Quintessons, while the Autobots were designed as consumer goods. Following the rebellion that forced the Quintessons off the planet, the Decepticons - as they named themselves - lusting for power, began a civil war. The Autobots could not hope to match the superior firepower and battlefield powers of the Decepticons, and instead turned to stealth, developing the art of transformation, modifying their bodies so that they could assume other forms. With this additional power, the Autobots were able to win the conflict, and a period of peace began, known as the Golden Age of Cybertron, when energy was plentiful and the planet shone with a golden hue.

Working in secret, the Decepticons also developed transformation technology, along with new robot-mode flight powers, and, under the command of one of the first of this new breed, Megatron, they attacked one of the capital's outer cities, killing the current Autobot leader. As the knowledge of this act was not made public, younger robots continued to idolize these powerful new flying robots - until one such young robot, Orion Pax, was duped by Megatron into allowing him access to an energy storage facility. Megatron turned on Pax and fatally injured him, but the ancient Autobot, Alpha Trion, then rebuilt Pax into the first of a new, battle-hardy breed of Autobot - now, he was Optimus Prime, leader of the Autobots, who led them against Megatron and the Decepticons as the civil war erupted once more.

After five million years, Cybertron was nearly depleted of all its energy when the Decepticons received word that the Autobots were planning to search for new energy resources. Following the Autobot ship, the Ark, in their ship, the Nemesis, the Decepticons boarded the Ark and engaged the Autobots on their ship. Unfortunately, the Ark drifted too close to Earth and crashed landed in a volcano. All the occupants of the Ark went into emergency stasis, while the Decepticons on the Nemesis ejected to South America. There, they scanned the forms of local insects, becoming the Insecticons. After 4 million years, the volcano the Ark was wedged in, erupted in Earth year 1984, restarting its computer, Teletraan I. Sending out Spy Satellites, Teletraan created alternative forms out of Earth vehicles for any transformers caught in its scanning beam (like F-15 Eagles for Skywarp and a Semi-trailer truck for Optimus Prime). The Decepticons awakened first but later the Autobots did too, thanks to Starscream's carelessness.

By the Earth year 2005, however, the Decepticons had succeeded in completely conquering Cybertron. In a final attempt to destroy the remaining Autobots they attacked Autobot City on Earth but were defeated. On their way back to Cybertron Astrotrain encountered difficulty leaving Earth's gravity due to an over-large payload. Starscream convinced the others to jettison all the damaged and dying decepticons, including Megatron. Megatron was then found by Unicron who offered to rebuild him in return for destroying the Autobot Matrix of Leadership. Megatron was reformed into Galvatron who returned to Cybertron, destroyed Starscream for his treachery and reassumed command of the Decepticons. Galvatron then attempted to use the Matrix against Unicron, provoking him into attacking Cybertron. The attack coupled with the temporary loss of Galvatron shattered the decepticon forces, allowing the Autobots to reclaim their home planet. The Decepticons fled to the planet Chaar, a world of ash and ruin, where they struggled to survive until Galvatron returned to them and reorganized them into a fighting force, battling to take back Cybertron.

Dreamwave Productions
After Autobot leader Optimus Prime and Decepticon leader Megatron disappeared in a Space Bridge accident several million years ago, the Autobots and Decepticons split up into several factions. One of those who broke away was Ratbat, who quickly took the opportunity to form his own power base. Gathering loyalist followers and setting up their HQ in the Polyhex region of Cybertron the Ultracons quickly came into conflict with the Autobot splinter faction The Wreckers. However, Ratbat added a secret weapon to his ranks - the combiner team the Constructicons, in flagrant defiance of treaties banning their use in the civil war as both Autobots and Decepticons splintered.

When the Ultracons attacked the Tagan Heights in order to seize its factories for themselves, they were opposed by the Wreckers. Ratbat quickly unleashed the Constructicons in their combined form of Devastator, routing the Wreckers. However, Ratbat was prevented from executing his rival Springer by the arrival of the Protectobots in their combined form of Defensor. The Autobot combiners were hoping to prevent an arms race between the various factions by stopping combiners signing up with the factions. After the battle devastated the very manufacturing facilities the two factions hoped to acquire, the Protectobots separated and defeated Devastator via sabotage. With their most potent weapon gone, the Ultracons retreated.

Later, the Ultracons joined with the Decepticons under Shockwave to support a peace accord with the Autobots, fighting off Starscream's Predacon faction. However, Megatron's return with an army of Seeker clones would bring a halt to the peace proceedings, and the Ultracons would be reabsorbed into the Decepticons.

IDW Publishing
The origins of the Decepticons in the IDW continuity would be explored in The Transformers: Megatron Origin, which confirmed Megatron as their founder in this timeline. A lowly miner, Megatron penned a manifesto against Functionalism, the forcible assignment of cybertronians to jobs based on their vehicle modes, which was in force at the time on religious grounds. Megatron began to work against the practice through peaceful protest, but following imprisonment and a brutal beating by police, he determines that force is the only way to bring about equality. Ironically, on his way out the station after having been saved from a government ordered 'accidental-death-by-interrogation', he first meets the future Optimus Prime, who would go on to use his original writings as the basis for the Autobot movement.

On the run after an unrelated attack on an Autobot Senator after a mining dispute, Megatron stumbled onto underground gladiator games in the state of Kaon and quickly became one of its rising stars. However, it only truly became more than this when Ratbat, then a senator seeing an opportunity for profit, dispatched Soundwave to arm the gladiators. Megatron's forces then began to commit acts of terrorism increasing in scale and daring, attracting the attention of Sentinel Prime. Kidnapping Senator Decimus as a symbol (with the help of new airborne recruit Starscream) Megatron issued a rallying call to the gladiators and their supporters, promising to unite them under one badge and make the whole planet their arena. Sentinel Prime's forces then found and arrested them en masse. This was all part of Megatron's plan: a supposedly informant Starscream massacred the Autobot Senate and freed the prisoners, signalling revolution and anarchy in Kaon. Megatron eventually seemed to have killed Sentinel Prime one-on-one, and the Decepticons were forged.

In this continuity, there are a number of differences to the traditional norm. The Decepticons are no longer based on Cybertron due to its devastation by rogue scientist Thunderwing. Instead, they have spread out through the galaxy, using their transforming abilities to infiltrate target worlds (such as Earth or Nebulos) disguised as local vehicles and equipment. They then use their modes to create conflict, leading to complete anarchy as the world's political and social systems collapse. The Decepticons then move in and harvest the world's raw resources. How the world is pacified is unknown, although The Transformers: Spotlight issue on Sixshot revealed he is often sent in to complete the destruction of entire worlds. As in other continuities, Megatron is in command, and thus far has had to deal with the rebellion of Starscream's Earth infiltration unit and the reemergence of Thunderwing, as well as moving on to Earth in order to harvest the extremely powerful Energon variant Ore-13, something which promises to give the Decepticons a significant edge over the Autobots.

The IDW continuity is unique in that it portrays the Decepticons as originally a force for good, standing up against oppression, who eventually themselves became oppressors. A large number of those who would become Autobots, including Optimus Prime himself, even support the Decepticon cause in the beginning, before it turns violent.

Beast Era
The Decepticons eventually evolve into the Predacons, a faction in the fictional Beast Wars universe created by Hasbro. Previously the name Predacon had been associated with the Transformer group which combined to form Predaking. The faction made their first appearance in the Beast Wars toyline and animated series, and have been represented in various Transformers animated series, comic books and toylines since.

A Predacon is most commonly represented as a Decepticon descendant with a new smaller, more energy-efficient body. This body transforms into an animal instead of a vehicle. Though the Decepticons and Predacons share many similarities, several important differences exist, such as leadership and ideals (for example, choosing a pretense of peace with the Maximals, rather than maintaining a war at all costs). It is not completely known as to how the faction gained predominance over all the other various Decepticon factions, although the comic book miniseries Beast Wars: The Gathering has provided hints that their heritage may be found in the original combiner team Predacons, hints confirmed by the subsequent Timelines: Dawn of Futures Past, which confirmed that Razorclaw, Divebomb and the other Predacons had been working behind the scenes.

Transformers: Robots in Disguise
In Transformers: Robots in Disguise, the Decepticons were not the name of the evil faction, but rather a sub-faction created by Megatron. The main faction was instead the Predacons (carrying the name of the main villains over from Beast Wars), who transformed into mechanical animals. Realising that their ability to travel as Earth vehicles gave the Autobots an edge in stealth, Megatron decided to take drastic action.

Megatron recovered the protoforms of six Autobots that had crashed to Earth some time before looking for Fortress Maximus. Scanning them with vehicle modes at a military base and infusing them with a fraction of his spark energy to ensure their new loyalty, the first five became the Commandos. But when Megatron tried to scan a tanker truck as an alternate mode for the leader Scourge, Optimus Prime was scanned as well, resulting in much of Prime's personality being scanned as well. With an infusion of Megatron's spark energy to complete the concoction, Scourge was born, emerging from his pod as a dark twin of Optimus Prime, who proved that Megatron's programming had over-ridden any Autobot goodness within him. Appointed leader of the new "Decepticons", they quickly established themselves as a force to be reckoned with.

The newly created Decepticons would try many schemes over the course of the series, including attacking various dams and faking their defection to the Autobots. But their leader, Scourge, had grander ambitions in mind. The copy of Optimus' personality had been warped by Megatron's evil influence, but the individuality of Optimus remained and Scourge now wanted to control both Predacons and Decepticons. He would seize his chance when Fortress Maximus was awakened, directing him to destroy Galvatron (Megatron's new form). The attempt failed and Galvatron took a terrible revenge - he stripped the Decepticons of their individuality, turning them into little more than drones. After Galvatron's defeat by Omega Prime, the Decepticons were among those sent back to Cybertron for imprisonment, except for Sky-Byte.

Unicron Trilogy
The Decepticons appeared in Transformers: Armada, Transformers: Energon, and Transformers: Cybertron as mostly jets and military vehicles.

In the Unicron Trilogy continuity, the early history of the war is not expounded upon. The Decepticons, however, remain the main antagonists, with whom the Autobots have been warring for quite some time. The enslavement of the small, power-enhancing Mini-Cons has long been a major goal of the Decepticons, and the Mini-Cons are seen as key to winning the war.

Near the end of Armada, the Autobots and Decepticons join forces to defeat the planet-devouring Unicron, and the beginning of Energon sees them as uneasy allies. Many return to Megatron's side when he returns.

Cybertron sees a much smaller Decepticon force than the previous two series, and what has become of most of the survivors of the earlier series remains unknown. As Megatron's true plans are discovered, most of the Decepticons, unwilling to be sacrificed as pawns as their leader seeks to remake the universe, again battle a common foe alongside the Autobots. The end of the series sees three of the comic-relief characters, with a reluctant Thundercracker, desiring to begin a new Decepticon force, and take off in a makeshift rocket. It makes it as far as Mars before it crashes, as the ending credits montage of the final episode shows. The foursome survives, however. The fates of the other surviving Decepticons remains unknown, though evidence of Starscream's survival is discovered.

Live-action films
The Movie Prequel comic by IDW Publishing would reveal their origins. Originally Optimus Prime and Megatron ruled Cybertron together, due to the power of the Allspark, a mysterious device that could give life to Transformers - and keep Cybertron itself alive. Eventually, Megatron began to desire its power for himself and assembled a like-minded army of separatists - the Decepticons. Megatron soon struck, and Cybertron became embroiled in civil war. However, this would later seem to be changed by the sequel films, where it is revealed that Sentinel Prime was the first leader of the Autobots and The Fallen the founder and supreme master of the Decepticons. Optimus would only take command after Sentinel was seemingly killed in action and Megatron served as leader of the second incarnation of the Decepticons with The Fallen as his master and mentor due to the latter's incapacitation. The subsequent prequel novel Transformers: Ghosts of Yesterday revealed that Starscream seized command of the Decepticons after Megatron's disappearance.

Transformers (2007 film)
The Decepticons make an appearance in the 2007 the Transformers live-action film. They include Megatron (Hugo Weaving), Starscream, Frenzy, Blackout, Barricade, Scorponok, Brawl and Bonecrusher. Like the Autobots, the Decepticons are designed to look more organic in the movie. While the Autobots have more human-like appearances, the Decepticons are more monstrous; they tend to resemble carnivorous insects, mammals and birds. In the film, the Autobots intend to use the AllSpark, the object that created their robotic race, in an attempt to rebuild Cybertron and end the war, while Megatron and the Decepticons desire to control it with the intention of building an army by giving life to the machines of Earth. Ultimately, nearly all the Decepticons who participated in the first battle (except Starscream, Scorponok and Barricade) are killed and the AllSpark is used by Sam to kill Megatron, thus making the Autobots accept Earth as their new home, while allying with humans to continue to hunt down remaining Decepticons.

Transformers: Revenge of the Fallen (2009 film)
In Transformers: Revenge of the Fallen it is revealed that The Fallen, a Prime who turned to evil, is the founder of the Decepticons, with Megatron being the leader of its second incarnation, accepting The Fallen as his master and mentor. This would seem to imply the Decepticons were originally Autobots who turned evil. In the final battle in the second movie, there are many Decepticon casualties including Devastator (destroyed by a rail gun), Mixmaster (cut in half and his head stomped off by Jetfire), Rampage (killed by Bumblebee), Ravage (stripped from his spine by Bumblebee), Scrapper and Long Haul (killed by an airstrike), Scorponok (head crushed by Jetfire) and the Fallen himself (spark ripped out by Optimus Prime). Also killed in the movie are Sideways (while still in his vehicle mode) by Sideswipe and Demolishor, killed at the beginning by Optimus and Ironhide. Following the Fallen's death, Megatron officially takes full command of the Decepticons at this point, swearing vengeance.

Transformers: Dark of the Moon (2011 film)
The Decepticons return in Transformers: Dark of the Moon with the plan to use a Space Bridge to bring an army to Earth and transport Cybertron into orbit to rebuild the planet with Earth's natural resources and human population as slaves. Megatron and Sentinel Prime planned this a long time ago, planning to meet on Earth before Megatron went after the Allspark and ended up frozen and the Ark was shot down by Decepticon fighters (few Decepticons knew of the plan) and crashed on Earth's Moon in 1961. This was detected by the Americans and the Russians and the space programs were created to reach the ship and study it. However, by 1963 the Decepticons had raided the Ark and removed all but five of the pillars and hid them on the Moon along with an army of hundreds of Decepticons, awaiting the time that the pillars would be used. After humans successfully reached the Moon and explored the ship, Soundwave and Laserbeak approached various humans including Dylan Gould's father to get them to shut down the American and Russian space programs.

The Decepticons established a network of human collaborators who kept the humans from noticing the Decepticons actions on the Moon, later led by Dylan Gould. After a human collaborator named Alexi Voshkod leads NEST to Chernobyl where they retrieve an engine part from the Ark, the Decepticons reemerge for the first time since Egypt in the form of Shockwave and his Driller who attack, but retreat. This leads the Autobots to retrieve Sentinel and the remaining space bridge pillars and revive him and as this is going according to plan, Megatron orders Laserbeak to kill all the human collaborators but Dylan Gould which he does. Before he's killed, one of the collaborators named Jerry Wang alerts Sam Witwicky to this and he alerts NEST which ignores it. Sam and Seymour Simmons find out about the Decepticons plot from pictures a Russian probe got of the Decepticons hiding the pillars and the Dreads are sent to attack the convoy protecting Sentinel, but are killed by Bumblebee, Dino, Sideswipe and Ironhide.

After Sentinel reveals his true colors and murders Ironhide, he sets up the pillars in the National Mall and brings the Decepticon army consisting of hundreds of Decepticon soldiers and Decepticon ships to Earth. With the army in hand, the Decepticons promise to bring no war against the human population in exchange that the Autobots must leave Earth. While taking off into the atmosphere, Starscream destroyed the ship, knowing that mankind will resist on their own. Some of the Decepticons scatter around the world to set up the pillars, but most of the army invades Chicago, devastating the human population and taking over. The Autobots, NEST and a team of mercenaries led by Sam Witwicky and Robert Epps invade Chicago and battle the Decepticon army, aided by the United States Army, and U.S. Air Force. The group manages to take out much of the army including Laserbeak (killed by Sam Witwicky and Bumblebee), the Driller (killed by Optimus), Starscream (killed by Sam Witwicky and Sergent Lennox), Soundwave (killed by Bumblebee), Barricade (killed by NEST) and Shockwave (killed by Optimus) and disrupt the Control Pillar, forcing Sentinel into battle. The combined human and Autobot forces overwhelm Sentinel and his remaining forces and Tomahawk missiles destroy most of the Decepticon fighters, forcing Sentinel to flee before engaging Optimus one-on-one. Dylan Gould reactivates the space bridge, but Sam Witwicky kills Gould, and Ratchet and Bumblebee destroy the Control Pillar, causing Cybertron to collapse and ruining the Decepticon plan. Also, the remaining Decepticon ships are sucked up into Cybertron by the space bridge and are destroyed with the planet.

Sentinel nearly kills Optimus, but Megatron, convinced by Carly Spencer that he would end up working for Sentinel unless he could get rid of him, turns on Sentinel and severely injures him. Upon seeing Cybertron being destroyed with the space bridge, Megatron attempts to make a false truce with Optimus, but Optimus doesn't fall for it and kills Megatron and Sentinel. While the loss of human life in Chicago is massive, the battle is a major victory for the Autobots as they eliminated most of the Decepticon army on Earth (including Gould by traitor) despite there being some still scattered around the globe and the entire Decepticon command structure was destroyed with all of its members killed in the battle.

Transformers: Age of Extinction (2014 film)
While the regular Decepticons do not actually appear in Transformers: Age of Extinction (except Megatron who was reborn as Galvatron), they do play a role in the story as secondary antagonists. The "Battle for Chicago" (climactic battle of Dark of the Moon) has become a 9/11 like event to the public 5 years after the third film and a ruthless government organization named Cemetery Wind have begun hunting Decepticons who are still scattered around the globe and secretly hunted Autobots alike when the alliance between humans and Autobots ended (including alliance between humans and Decepticons as Dylan Gould is already dead). It was mentioned that there might still be less than a dozen Decepticons and Autobots hiding on the planet Earth by Harold Attinger (Kelsey Grammer). The film's antagonist is instead the neutral assassin and bounty hunter Lockdown, who may have been a Decepticon at one point, sent to Earth to hunt Optimus Prime by the Transformers' creator race (unseen in the movie). Lockdown and Attinger make a three-way deal along with a company named K.S.I., Kinetic Solutions Incorporated, by the boss, Joshua Joyce, who uses the original Decepticons' (and Autobots') remains to build new Transformer prototypes which can be controlled by humans and must eliminate those who cross their path such as the Yeager family who know this secret.

Their crown jewel is Galvatron, who becomes possessed by Megatron's (Frank Welker) consciousness and then possesses the other prototypes and turns them into Decepticons controlled by him. Galvatron plans to use the Seed, a device that can create Transformium, the metal Transformers are made of, to rebuild his Decepticon army and conquer Earth, when Joyce was called by Cade Yeager, and, knowing that he was manipulated by Attinger and Galvatron, escapes with the Seed. After the final battle in Hong Kong, all the Decepticons are destroyed by Cade, the Autobots and Dinobots, except Galvatron, who escapes after Lockdown and Attinger are killed by Optimus Prime, vowing to return one day, for he is reborn, before Cemetery Wind dissolved by their crimes and branded as a terrorist organization.

Barricade, Soundwave and a number of other Decepticon "protoform drones" appear in archival footage from Dark of the Moon presented by Attinger. Various deceased Autobots and Decepticons appear throughout the film in Chicago and in the KSI lab, including Megatron's head.

Transformers: The Last Knight (2017 film)
The Decepticons return in Transformers: The Last Knight with the revived leader, Megatron, when a war for survival has commenced between the human race and the Transformers and set up a military organization of TRF (Transformers Reaction Force). The Decepticons began to search for the staff when Barricade reports to Megatron that human has found the mysterious talisman they're seeking and suggests using TRF their purposes. Megatron contacting TRF abducting two CIA agents and offered an exchange to release his group of Decepticons. During the final battle, they assist the sorceress Quintessa in draining the life-energy from Earth to repair a damaged Cybertron and began to defend the from the Autobots and military. But the Autobots broke through and most of the Decepticons began to die and were defeated.

Bumblebee (2018 film)
The Decepticons return in Bumblebee, where they are first seen during the final battle for Cybertron against the Autobots. The Decepticons on Cybertron are co-led by Shockwave and Soundwave, who proceed to force the Autobots off of Cybertron.

At some point prior to this battle, the Decepticon Seeker Blitzwing was sent to Earth by the Decepticons for unknown reasons. After the Autobot scout B-127 lands on Earth, Blitzwing intervenes in an ensuing skirmish between the Autobot and the human special operatives group known as Sector Seven. He proceeds to engage B-127 and interrogate him about Optimus Prime's whereabouts, even ripping out the scout's vocal processor and attempting to execute him. Before he could do this, however, B-127 attached one of Blitzwing's missiles onto Blitzwing's body, causing him to explode and die in the process.

Decepticon Triple-Changer agents, Shatter and Dropkick are sent to intercept lieutenant Cliffjumper, one of the Autobots that escaped at the beginning of the film, in the hopes that they may find the Autobot leader, Optimus Prime. Cliffjumper refuses to reveal Optimus' whereabouts and thus subsequently fatally sliced in half by Dropkick.

Transformers Animated
In Transformers Animated, the Decepticons are usually bigger and stronger than the Autobots and usually turn into jets and military vehicles. It takes a group of Autobots to defeat one Decepticon and almost all Decepticons have red eyes and can fly[citation needed]. Most Decepticons in the series are similar to the original G1 series. In the Animated Series Blitzwing, for example, retains his triple changer modes of a tank and jet and also retains his G1 tan and purple color scheme.

This series depicts a Decepticon force that is far from what it apparently once was, with Megatron having been chasing the Allspark for millions of years prior to the series' beginning. The Decepticons have been "wandering the periphery" for some time, but have been driven off of Cybertron long ago. These Decepticons see themselves as "freedom fighters", seeking to throw off "Autobot tyranny", and their plans tend to revolve around returning to Cybertron with the means to retake it. What started the great war in this series was some of the Cybertronians being kicked off of Cybertron for being freaks, eventually forming an alliance called the "Decepticons". Now the Decepticons fight for their rights and try to take back what was rightfully theirs.

Transformers: Timelines
A series of stories printed by Fun Publications, set in a variety of continuities, the Decepticons of alternate worlds can vary greatly from the classics Decepticon image. In the Transtech storyline, the Autobots and Decepticons are merely political factions on a Cybertron which never broke out into war. Lines between good and evil are more blurred.

In the Shattered Glass storyline, a group of Decepticons, led by Megatron, are the mirror images of the personalities of most Decepticons. In this world the Decepticon logo is red, and worn by heroic Transformers who seek to protect the weak and oppressed from the evil Autobots.

Aligned Continuity
The main group of Decepticons, led by Megatron, in the 2010 computer animated series Transformers: Prime is much larger than in most continuities considering their ranks are bolstered by a clone army of drone named Vehicons (a possible homage to Beast Machines) who all look-alike though some have different vehicle modes and have personalities. The Vehicons are sometimes the center of a joke often shown unable to do their job and are very incompetent as well as constantly being killed off way too easily by the Autobots. However The Decepticons don't appear in the series Transformers: Rescue Bots, though it has been heavily implied that the two shows take place in the same continuity. The Decepticons are briefly mentioned by Heatwave in the episode "Bot to the Future", (who also mentioned Unicron in an earlier episode) when the Autobots arrive in an alternate timeline ruled by Morbots (who resemble Vehicons), Bumblebee believes the Morbots are Decepticons, only for Heatwave to correct him. In a later episode, the recruits retrieve what appears to be a cassette tape from a derelict spaceship and it transforms into an injured Laserbeak. The group disagree over whether to help him as he is a Decepticon (Hotshot in particular is against doing so) and it's ultimately revealed that Wedge was originally built as a Decepticon but was given the chance to instead be an Autobot (a fact he kept from the others due to the prejudices shown in this episode). In the end, they allow Laserbeak to stay at the academy while he recovers.

Transformers: War for Cybertron
In Transformers: War for Cybertron and Transformers: Fall of Cybertron, Megatron has come across a legendary element called "Dark Energon". He infiltrates Starscream's space station in order to corrupt himself with it. He then kills the Autobots' leader Zeta Prime and obtains the Omega Key, which then activates Omega Supreme. After Megatron defeats Omega Supreme, he goes to Cybertron's core and corrupts it with Dark Energon. Optimus Prime becomes the new Autobot leader, and heads into the planet to rescue Omega Supreme. He opens the Omega Lock into the core after Ratchet purges the Dark Energon corruption. Optimus discovers that he is too late to save Cybertron from the Dark Energon corruption, and the core must shut down for about one million years to survive. Optimus is giving a piece of Cybertron's spark for safe-keeping (The Matrix of Leadership). When Optimus evacuates the Autobot forces from Cybertron, Megatron tries to stop them from leaving. Silverbolt, Air Raid, and Jetfire try to stop it, but the space station transforms into Trypticon and crashes on Cybertron. Optimus and his Autobots then defeat Trypticon. Megatron has Trypticon rebuilt into the Nemesis. The Nemesis pursues the Ark, and then Megatron shoots Bumblebee. They go into a space bridge, and leave Cybertron to a lifeless wasteland.

Transformers: Prime

This article's plot summary may be too long or excessively detailed. Please help improve it by removing unnecessary details and making it more concise. (December 2019) (Learn how and when to remove this template message)
By the time of Transformers: Prime, it has been three years since the Decepticons last attacked Earth and the Autobots still await their return. After the death of Cliffjumper, the Autobots fight to protect the Earth from the Decepticons and befriend three young humans while preparing for the return of Megatron, who has been missing for three Earth years. Megatron plans to use Dark Energon to raise an undead army of Cybertronians and defeat the Autobots. The Autobots destroy his space bridge and Megatron is believed killed in the process. Starscream becomes the new Decepticon leader. After Megatron's apparent death, new Decepticon leader Starscream does not stray from Megatron's path and recruits several new Decepticons. Starscream attempts many missions to destroy the Autobots and find their headquarters, while keeping a shard of the "last" remaining Dark Energon that he took out of Megatron's chest. In the episode "Out of His Head", Megatron returns after an incident where he takes control of Bumblebee's mind. Megatron then reclaims leadership of the Decepticons, keeping a strict eye on Starscream. After Starscream uses Megatron's share of Dark Energon, he tries to find more to once again bring back his "un-dead army". At the end of the episode "Partners", Starscream became an independent, striking out on his own, having become fed up with being disrespected. During the final four episodes of the season, the Autobots unwillingly team up with Megatron to battle a legendary threat to Earth's existence, Unicron.[3] To defeat Unicron, Optimus uses the Matrix of Leadership.

With this sacrifice, he not only loses the Wisdom of the Primes but also his memories of the war and his comrades, and joins Megatron as a Decepticon under his old name, Orion Pax. Orion, under the false truths about the Autobots being evil and the Decepticons being good, Megatron, then sets about decoding the Iacon Database. The Autobots save him, journeying to Cybertron and back to recharge the Matrix of Leadership and help him regain his memories, but the Decepticons continued to search for the relics Orion did decode throughout Season 2, managing to decode all of them thanks to Soundwave's persistence and computer skills. Starscream meanwhile attempts several times to create personal armies and an alliance with MECH, only for the human terrorists to take his T-cog to construct Nemesis Prime as well as a failed attempt to assassinate Megatron. The Insecticons also join the crew of the Nemesis when Airachnid tries to eliminate Megatron for trying to terminate her previously. Airachnid is later imprisoned in a Stasis pod by Acree and held as a prisoner in the Autobot Base. Team Prime also is granted another Autobot when Smokescreen, the rookie warrior and bodyguard of Optimus Prime's mentor Alpha Trion, arrives on Earth in a Decpticon escape pod and Megatron gains MECH leader Silas, who has taken Breakdown's body for his own after nearly being killed by Optimus Prime, as an ally until he fails Megatron's expectations, causing Cylas (as he calls himself) to end up as Knockout's science experiment. When Optimus claims possession of the Star Saber and learns of the last four Iacon relics: the Omega keys (devices that can restore Cybertron), Megatron replaces his arm with an Ancient Prime's arm, so he can use the Forge of Solus Prime to turn a chunk of Dark Energon into the Dark Star Saber. In the battle that follows, Megatron destroys the Star Saber, robbing Optimus of Victory.

Defeated, Optimus Prime retreats, gaining the first Omega Key in the process due to Smokescreen's clever thinking. Knock Out obtains the second however using the Resonance Blaster, and Starscream obtains the third using a dose of speed-enhancing Red Energon. The fourth and final one is revealed by Soundwave to have been inside Smokescreen, placed there to make sure it got to Optimus from Alpha Trion. After the Decepticons kidnap him, Knock Out uses the phase shifter to remove it. Megatron also finds out the purpose of the keys, by using the cortical psychic patch on Smokescreen. When Megatron leaves Knockout alone with him, Smokescreen and Knockout fight over the phase shifter, ending with the Autobot placing Knockout in a wall and leaving him there. He takes the one the Decepticons had and his and escaped the ship, starting a free fall battle, resulting in Smokescreen's escape. Starscream then goes on to steal the three keys the Autobots had. Starscream, now with all 4, uses them as a peace offering to rejoin the Decepticons.

Dreadwing becomes angry that Megatron allowed Starscream to rejoin the Decepticons after seeing that Starscream turned Skyquake into a Terrorcon. He gives Optimus the Forge of Solus Prime, and attempts to kill Starscream but is killed in the process by Megatron. Knockout and Starscream use the keys and find out the location of the Omega Lock, with Optimus already knowing its location and turning the Goundbridge into a spacebridge, and reforging the Star saber. The Autobots head to Cybertron, fighting off the Decepticons with their Iacon relics, take back the keys and manage to kill all the Vehicons, managing to reach the Omega Lock. Sadly, the victory is short lived, as the Decepticons force them to give them back the keys, or they will expose Jack, Miko, and Raf to Cybertron's toxic atmosphere. The Autobots are forced to yield and give them the keys, which Megatron uses to rebuild the Hall of Records. He then opens a space bridge, and tries to terraform Earth into a new planet he considers calling New Kaon.

Optimus, fearing for humanity and unwilling to let another world be destroyed, takes his Star Saber and cuts off Megatron's arm and destroys the Omega Lock, leaving Cybertron in its liveless state forever. The Autobots retreat back to Earth, only to find that Jasper, Nevada has now been changed into a giant fortress. The Nemesis lands there, and Optimus discovers the Decepticons have discovered the location of their base. Under Megatron's orders, Starscream leads an armada of Vehicons and Insecticons to start attacking the base. Wheeljack and Agent Fowler try and hold off the new Vehicons as long as they are able, trying to allow Team Prime to escape. Optimus has Team Prime split up to different parts of the country in the hope of making them more difficult to hunt down, with Optimus remaining behind to ensure the Decepticons could not follow. Outside, Wheeljack's ship, the Jack Hammer is shot down by Starscream. The Decepticons then blow up the base with Optimus inside as he destroys the Ground Bridge with Agent Fowler and June Darby watching in horror as the base explodes. Megatron and Starscream then fly down to the remains of the base to search for survivors. Celebrating their victory, they fail to notice Optimus's arm sticking out of the rubble.

In Season 3, the Decepticons hold earth hostage with their fortress, christened as Darkmount, and search for the autobots across the United States, To track the autobots, the recently returned Shockwave brings with him a Predacon to hunt down the scattered Autobots. When the Predacon fails, the search continues under Starscream's command. This search is put on hold however when the Autobots unite under Ultra Magnus to assault Darkmount. However, as the assault continues, it is the Decepticons who emerge victorious even with the loss of the Predacon, capturing Team Prime. However, Optimus Prime, returned to health by Smokescreen's use of the Forge of Solus Prime, attacks Darkmount, defeating Megatron in one on one combat in the air and destroying the fortress' fusion cannons so that the military can destroy the fortress. Forced to use the Nemesis as their base once again, Megatron begins sending units to hunt down Predacon fossils in the hope of gaining an advantage over the Autobots through cloning an army. The return of the Predacon (Predaking himself) adds to this advantage. However, the Decepticon ranks are thinned when Starscream and Knock Out accidentally turn Cylas into a Terrorcon that can turn other Decepticons into Energon vampires through a mixture of Synthetic and Dark Energon. This plague also leads to Airachnid's escape and her retaking control of the Insecticons, forcing Soundwave to teleport her armada to one of Cybertron's moon's and leaving them to be eaten by her as she has become a Terrorcon hybrid. This accident forces the cloning process to be sped up on Project Predacon due to the loss of more than half of Megatron's forces. However, Project Predacon is terminated when Predaking reveals he is capable of higher thought as well as transformation, forcing Megatron to lure the Autobots into destroying the cloning laboratory for him. With the destruction of the lab, it is revealed that Predacon CNA combined with Synthetic Energon is capable of producing cybermatter. Attention is then focused on the restoration of the Omega Lock and the completion of the Synthetic Energon formula. To complete it, Ratchet is kidnapped and forced to aid in completing the formula, at which he succeeds. Megatron however does not count on Ratchet turning Predaking against him, nor the Autobots assaulting the Nemesis. With this, Megatron rallies his troops for a final stand. Though he kills Bumblebee and nearly kills Optimus, Megatron is killed by a resurrected Bumblebee who emerges from the Omega Lock and stabs him in the spark with the Star Saber, sending him tumbling to the earth far below. With Megatron's death, and Soundwave trapped in the Shadowzone by Jack and Miko, the remaining Decepticons are killed, imprisoned or retreat with Shockwave and Starscream to the ship's escape pods.

In the film "Predacons Rising" - which serves as the series finale - it is further revealed that while many Decepticons were imprisoned, some defected to the Autobots to help rebuild Cybertron. Starscream and Shockwave, however, remain at large and having cloned two new Predacons they have named Skylynx and Darksteel, they continue to hunt Predacon bones to clone more soldiers. When Megatron returns, Starscream is at first overjoyed to have their leader back, but upon realization that Unicron is overshadowing and controlling him, Starscream retreats while Shockwave and the Predacons fight. Shockwave is also the one who suggests that the Predacons, now under Predaking's full leadership, help the Autobots defeat Unicron and his undead army. Starscream and the Decepticon prisoners aboard the Nemesis attempt to take control of the ship, only for Knock Out to change allegiance to the Autobots and imprison Starscream while the remaining mutineers are killed. Though Starscream escapes, Knock Out stays to help stop Unicron. When Unicron is defeated and imprisoned by Optimus Prime, Megatron is freed from his control and officially disbands the Decepticons and flies off to start a new life, as he is tired of conquest and no longer wishes to oppress anyone, having just experienced it himself. Starscream takes Megatron's throne but his rule is short-lived when the Predacons arrive to settle old scores, the outcome of which remains unknown.

Transformers: Robots in Disguise
Five years after the events of "Predacons Rising", Bumblebee is alerted to the existence of a new group of Decepticons on Earth. He eventually assembles a team-consisting of Autobots Strongarm, Sideswipe, Grimlock, and Fixit-to contend with these new Decepticons, which include the likes of Underbite and Bisk. Steeljaw, in particular, is the main antagonist, leading a group of Decepticons consisting of bounty hunter Fracture, crime boss Thunderhoof, the metal-consuming Underbite, and the crab-like Clampdown, in hopes of giving the Decepticons their own world. Towards the end of season one, Megatronus/The Fallen, the first Decepticon, contacts Steeljaw and convinces him to build him a space bridge so he can return to Earth. Steeljaw does so, but once on Earth, Megatronus attempts to destroy both Earth and Cybertron. Steeljaw tries to stop him, but Megatronus uses his telepathic powers to throw him into the distance. Ultimately, Megatronus is killed by Bumblebee's team and the rest of Steeljaw's pack is imprisoned.

In season two, Steeljaw comes across a group of Decepticons who have turned the Alchemor into their own base, enslaving the Mini-Cons there as well. These Decepticons are led by the strict Glowstrike, ex-pirate Sabrehorn, and the gruff Scorponok. Steeljaw manages to reform his pack by using Scorponok as a distraction while he infiltrates the Autobots' scrapyard and frees Clampdown and Thunderhoof from stasis, along with the insane Springload, the revolutionary Quillfire, hyperactive Bisk, former gladiator Groundpounder, and spy/saboteur Overload. Scorponok is arrested by Bumblebee's team, with Steeljaw fabricating a story that Scorponok had betrayed them. At the end of the season, Steeljaw usurps Sabrehorn and Glowstrike and becomes leader of the escaped Decepticons (collectively known as "Decepticon Island"), but Bumblebee's team sets off a specialized bomb that places all Decepticons on board the Alchemor in stasis, including Steeljaw. Optimus Prime, Windblade, Ratchet, and the former Mini-Con prisoners then take the recaptured Decepticons back to Cybertron while Bumblebee decides to establish a permanent Autobot base on Earth."""

In [10]:
additional_data = user.dataloader.dataset.tokenizer(text)
len(additional_data["input_ids"])

11724

In [11]:
512 * 22

11264

In [12]:
seq_length = 512

In [13]:
B = len(additional_data["input_ids"]) // seq_length
print(B)
data = torch.as_tensor(additional_data["input_ids"][:seq_length * B]).reshape(B, seq_length)
input_data = dict(input_ids=data, labels=data)
true_user_data = dict(data=input_data, labels=input_data)
server_payload["metadata"].shape = [seq_length]
server.secrets["ImprintBlock"]["data_shape"] = [seq_length]
user.num_data_points = B

22


In [14]:
shared_data, true_user_data = user.compute_local_updates(server_payload, custom_data=input_data)
shared_data["metadata"]["num_data_points"] = B

Computing user update in model mode: eval.


In [15]:
attacker.cfg.sentence_algorithm = "dynamic-threshold"

In [16]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

#user.print(reconstructed_user_data)

Recovered tokens tensor([    1,     1,     1,  ..., 50178, 50178, 50178]) through strategy embedding-norm.
Recovered 9660 embeddings with positional data from imprinted layer.
Assigned [369, 436, 441, 436, 448, 440, 448, 435, 432, 438, 438, 438, 443, 432, 493, 452, 446, 447, 427, 431, 444, 446] breached embeddings to each sentence.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/accuracy/3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Thu Jan  6 18:45:02 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/bleu/8bc44c0b5bba429a0a448e9968a8961bf9479d6ee30e00e93fed2c693040265a (last modified on Thu Jan  6 18:45:03 2022) since it couldn't be found locally at bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the

In [17]:
attacker.cfg.embedding_token_weight = 0.25
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

#user.print(reconstructed_user_data)

Recovered tokens tensor([    1,     1,     1,  ..., 50178, 50178, 50178]) through strategy embedding-norm.
Recovered 9660 embeddings with positional data from imprinted layer.
Assigned [369, 436, 441, 436, 448, 440, 448, 435, 432, 438, 438, 438, 443, 432, 493, 452, 446, 447, 427, 431, 444, 446] breached embeddings to each sentence.
Replaced 70 tokens with avg. corr 0.07406255602836609 with new tokens with avg corr 0.33675870299339294
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/accuracy/3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Thu Jan  6 18:45:02 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/bleu/8bc44c0b5bba429a0a448e9968a8961bf9479d6ee30e00e93fed2c693040265a (last modified on Thu Jan  6 18:45:03 2022) since it couldn

In [19]:
attacker.cfg.embedding_token_weight = 0.25
attacker.cfg.sentence_algorithm="k-means"
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

#user.print(reconstructed_user_data)

Recovered tokens tensor([    1,     1,     1,  ..., 50178, 50178, 50178]) through strategy embedding-norm.
Recovered 9660 embeddings with positional data from imprinted layer.
Assigned [512, 512, 512, 1, 1, 1, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 441, 512, 512, 512, 512] breached embeddings to each sentence.
Replaced 69 tokens with avg. corr 0.06986384093761444 with new tokens with avg corr 0.31250301003456116
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/accuracy/3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Thu Jan  6 18:45:02 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/bleu/8bc44c0b5bba429a0a448e9968a8961bf9479d6ee30e00e93fed2c693040265a (last modified on Thu Jan  6 18:45:03 2022) since it couldn't be 

In [22]:
# attacker.cfg.embedding_token_weight = 0.0
# attacker.cfg.sentence_algorithm="dynamic-threshold"

In [25]:
# results_server = dict()
# for v_length in [32, 48, 64]:
#     for eps in [1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]:
#         for softmax_skew in [1e6, 1e7, 1e8, 1e9, 1e10, 1e11, 1e12]:
#             for sequence_token_weight in [10, 1, 0.1, 0.01, 1e-3]:
#                 for measurement_scale in [1e3, 1, 1e-3]:
#                     key = f"{v_length}-{eps}-{softmax_skew}-{sequence_token_weight}-{measurement_scale}"
#                     print(f"Checking {key}")
#                     cfg.attack.token_strategy="embedding-norm"
#                     cfg.case.server.param_modification.v_length = v_length

#                     cfg.case.server.param_modification.eps = eps
#                     cfg.case.server.param_modification.imprint_sentence_position = 0
#                     cfg.case.server.param_modification.softmax_skew = softmax_skew
#                     cfg.case.server.param_modification.sequence_token_weight = sequence_token_weight

#                     cfg.case.server.param_modification.measurement_scale = measurement_scale

#                     cfg.case.server.pretrained = False

#                     user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
#                     attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)

#                     server_payload = server.distribute_payload()

#                     B = len(additional_data["input_ids"]) // seq_length
#                     user.data_key = "input_ids"
#                     data = torch.as_tensor(additional_data["input_ids"][:seq_length * B]).reshape(B, seq_length)
#                     input_data = dict(input_ids=data, labels=data)
#                     true_user_data = dict(data=input_data, labels=input_data)
#                     server_payload["metadata"].shape = [seq_length]
#                     server.secrets["ImprintBlock"]["data_shape"] = [seq_length]
#                     user.num_data_points = B
#                     shared_data, true_user_data = user.compute_local_updates(server_payload, custom_data=input_data)
#                     shared_data["metadata"]["num_data_points"] = B
#                     try:
#                         reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
#                                                                       server.secrets, dryrun=cfg.dryrun)

#                         metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
#                                                             server.model, cfg_case=cfg.case, setup=setup)

#                         results_server[key] = metrics["accuracy"]
#                     except:
#                         results_server[key] = 0.0

Checking 32-1e-05-1000000.0-10-1000.0
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Loading cached processed dataset at /home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-50b83d69230e0003.arrow
Loading cached processed dataset at /home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-bc2379d774fe375e.arrow
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab64

Loading cached processed dataset at /home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-4ca0616f34218fbb.arrow
Loading cached processed dataset at /home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-5272a8058f42c1f4.arrow
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Loading cached processed dataset at /home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-6bac458fa753dfab.arrow
Loading cached processed dataset at /home/jo

Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Loading cached processed dataset at /home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-de2cbbcab2a94d0a.arrow
Loading cached processed dataset at /home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-964297ce93a09a12.arrow
Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is 0.047953084111213684, feature std is 0.7348622679710388.
Computing user u

Loading cached processed dataset at /home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-1e1757bbcdf94273.arrow
Loading cached processed dataset at /home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-b7363800a13aaf66.arrow
Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is 34.42696762084961, feature std is 1048.4576416015625.
Computing user update in model mode: eval.
Recovered tokens tensor([    1,     1,     1,  ..., 50178, 50178, 50178]) through strategy embedding-norm.
Recovered 9315 embeddings with positional data from imprinted layer.
Assigned [356, 412, 423, 438, 423, 416, 494, 415, 434, 421, 430, 418, 429, 417, 443, 416, 412, 421, 424, 420, 428, 425] breached embeddings to each sentence.
Using the latest cached version of the module from /home/jonas/.

Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is -1.484903805248905e-05, feature std is 0.0009906507330015302.
Computing user update in model mode: eval.
Recovered tokens tensor([    1,     1,     1,  ..., 50178, 50178, 50178]) through strategy embedding-norm.
Recovered 9482 embeddings with positional data from imprinted layer.
Assigned [467, 434, 424, 444, 443, 422, 446, 431, 422, 433, 435, 432, 425, 414, 412, 424, 426, 422, 444, 413, 433, 436] breached embeddings to each sentence.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/accuracy/3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Thu Jan  6 18:45:02 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/data

...

METRICS: | Accuracy: 0.5191 | S-BLEU: 0.17 | FMSE: 2.4109e-02 | 
 G-BLEU: 0.19 | ROUGE1: 0.47| ROUGE2: 0.20 | ROUGE-L: 0.44| Token Acc: 66.28% | Label Acc: 66.28%
Checking 32-1e-05-100000000000.0-0.1-1
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa7

Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/bleu/8bc44c0b5bba429a0a448e9968a8961bf9479d6ee30e00e93fed2c693040265a (last modified on Thu Jan  6 18:45:03 2022) since it couldn't be found locally at bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/rouge/2b73d5eb463209373e9d21a95decb226d4164bdca4c361b8dfad295ec82bc62e (last modified on Thu Jan  6 18:45:22 2022) since it couldn't be found locally at rouge, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/google_bleu/bd7e6c5887f75974a6272946388ed86b5be85e95466b9f71edd4bd0ff5041640 (last modified on Thu Jan  6 18:45:22 2022) since it couldn't be found locally at google_bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /ho

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/datasets/wikitext/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126 (last modified on Wed Jan 12 12:53:37 2022) since it couldn't be found locally at wikitext.
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Found attention of shape torch.Size([2304, 768]).
Computing feature distribution before the probe layer Conv1D() from external data.
Feature mean is 21.731966018676758, feature std is 1057.179931640625.
Computing user update in model mode: eval.
Recovered tokens tensor([    1,     1,     1,  ..., 50178, 50178, 50178]) through strategy embedding-norm.
Recovered 8974 embeddings with positional data from imprinted layer.
Assigned [419, 4

Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/rouge/2b73d5eb463209373e9d21a95decb226d4164bdca4c361b8dfad295ec82bc62e (last modified on Thu Jan  6 18:45:22 2022) since it couldn't be found locally at rouge, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/google_bleu/bd7e6c5887f75974a6272946388ed86b5be85e95466b9f71edd4bd0ff5041640 (last modified on Thu Jan  6 18:45:22 2022) since it couldn't be found locally at google_bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/jonas/.cache/huggingface/modules/datasets_modules/metrics/sacrebleu/015a527b2e180e7da27bda77c3fb309ebab15dba5d484fa6f3fec5e9f762c849 (last modified on Thu Jan  6 18:45:40 2022) since it couldn't be found locally at sacrebleu, or remotely on the Hugging Face Hub.
METRICS: | Accuracy: 0.4935 | S-BLEU: 0.15 |

KeyboardInterrupt: 